<a href="https://colab.research.google.com/github/aayushbhurtel/Data_Visualization/blob/main/Quiz15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>